In [ ]:
import robotic as ry
import numpy as np
import time

In [ ]:
config = ry.Config()
config.addFile("puzzles/p5-wall-easy.g")

In [ ]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

In [ ]:
config.view()
f = config.getFrame('camera_top')
config.view_setCamera(f)

In [ ]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [ ]:
def is_in_line_of_sight(point : np.array, frame : ry.Frame, config : ry.Config) -> bool:
    # TODO compute if there is a line of sight between the point and the frame
    return True

In [ ]:
def get_proximity_score(point : np.array, goal_frame : ry.Frame) -> int:
    distance = np.linalg.norm(point - goal_frame.getPosition()[:2])
    if distance < 0.2:
        return 5
    elif distance < 0.4:
        return 2
    else:
        return 0

In [ ]:
def get_subgoal_score(
        point : np.array,
        agent_frame : ry.Frame,
        goal_frame : ry.Frame,
        config : ry.Config) -> int:
    return 10 * int(is_in_line_of_sight(point, goal_frame, config)) + 5 * int(is_in_line_of_sight(point, agent_frame, config)) + get_proximity_score(point, goal_frame)